In [2]:
from galpy.potential import NFWPotential as NFWpot
from galpy.potential import HernquistPotential as HQpot
from galpy.potential import DoubleExponentialDiskPotential as DEDpot
from galpy.potential import MiyamotoNagaiPotential as MNpot
from galpy.potential import MN3ExponentialDiskPotential as MN3pot


In [ ]:
NFWpot.dens()